In [ ]:
from rmprocs.dm import * 
from rmprocs.suavizamiento import * 

import pandas as pd 
import numpy as np 

from matplotlib import pyplot as plt 
import seaborn as sns

In [ ]:
oDmApp, PROJECT_FOLDER = get_oDmApp()

def log(x): 
    x = x + "\n"
    oDmApp.ControlBars.Output.Write(x)

In [ ]:
table = "mb_final_smooth_r3a"

In [ ]:
bm = get_dm_table(table, oDmApp)
bm["VOL"] = bm["XINC"] * bm["YINC"] * bm["ZINC"] 

In [ ]:
def summarize_bands(df: pd.DataFrame, dist: float, level_thickness: float) -> pd.DataFrame:
    """
    Summarize KDTree batching bands by Z levels.

    Parameters
    ----------
    df : DataFrame
        Must contain columns ["XC", "YC", "ZC"].
    dist : float
        Neighborhood radius used for smoothing. Also used as overlap.
    level_thickness : float
        Core band thickness along Z.

    Returns
    -------
    summary : DataFrame
        One row per core band with:
        - band_id: sequential band index starting at 1
        - core_lo, core_hi: Z bounds of the core band [lo, hi)
        - z_center: midpoint of the core band ( (lo+hi)/2 )
        - cx, cy, cz: centroid (mean) of core-band cells (NaN if core empty)
        - n_indexed: number of cells in expanded band (core ± dist)
        - n_smoothed: number of cells in core band
    """
    if level_thickness <= 0:
        raise ValueError("level_thickness must be > 0")
    if "XC" not in df or "YC" not in df or "ZC" not in df:
        raise KeyError('df must contain columns "XC", "YC", and "ZC"')

    # Use float32 views for masks and any potential heavy ops
    z = df["ZC"].to_numpy(dtype=np.float32, copy=False)
    x = df["XC"].to_numpy(dtype=np.float32, copy=False)
    y = df["YC"].to_numpy(dtype=np.float32, copy=False)

    zmin = float(np.nanmin(z))
    zmax = float(np.nanmax(z))
    overlap = float(dist)

    # Build the same non-overlapping core windows you use in smoothing
    level_edges = []
    z_start = zmin
    while z_start <= zmax:
        z_end = z_start + level_thickness
        level_edges.append((z_start, z_end))
        z_start = z_end

    rows = []
    for li, (core_lo, core_hi) in enumerate(level_edges, 1):
        exp_lo = core_lo - overlap
        exp_hi = core_hi + overlap

        core_mask = (z >= core_lo) & (z < core_hi)
        exp_mask  = (z >= exp_lo)  & (z < exp_hi)

        n_smoothed = int(np.count_nonzero(core_mask))
        n_indexed  = int(np.count_nonzero(exp_mask))

        if n_smoothed > 0:
            # Centroid over core cells
            cx = float(np.nanmean(x[core_mask]))
            cy = float(np.nanmean(y[core_mask]))
            cz = float(np.nanmean(z[core_mask]))
        else:
            cx = cy = cz = np.nan

        rows.append({
            "band_id":   li,
            "core_lo":   float(core_lo),
            "core_hi":   float(core_hi),
            "z_center":  float((core_lo + core_hi) * 0.5),
            "cx":        cx,
            "cy":        cy,
            "cz":        cz,
            "n_indexed": n_indexed,
            "n_smoothed": n_smoothed,
        })

    return pd.DataFrame(rows)

In [ ]:
lt = 5
res = summarize_bands(bm, dist=10, level_thickness=lt)

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(10, 4))
sns.histplot(bm, x="ZC", binwidth=lt, ax=axs[0])
sns.lineplot(res, x="z_center", y="n_indexed", ax=axs[0])
sns.lineplot(res, x="z_center", y="n_smoothed", ax=axs[0])

In [ ]:
def summarize_bands_xyz(
    df: pd.DataFrame,
    dist: float,
    x_size: float,
    y_size: float,
    z_size: float,
) -> pd.DataFrame:
    """
    Summarize 3D KDTree tiling bands (X×Y×Z).

    Parameters
    ----------
    df : DataFrame
        Must contain columns ["XC", "YC", "ZC"].
    dist : float
        Neighborhood radius used for smoothing; also defines overlap in all axes.
    x_size, y_size, z_size : float
        Core tile sizes along X, Y, Z (must be > 0).

    Returns
    -------
    summary : DataFrame
        One row per core tile with:
        - tile_id : sequential tile index (1-based)
        - ix, iy, iz : tile indices along X, Y, Z (0-based)
        - x_lo, x_hi, y_lo, y_hi, z_lo, z_hi : core bounds [lo, hi)
        - x_center, y_center, z_center : centers of the core box
        - cx, cy, cz : centroid (mean) of core cells; NaN if core empty
        - n_indexed : rows in expanded (indexing) box (core ± dist in all axes)
        - n_smoothed : rows in core (write zone)
    """
    # --- Validation ---
    if not all(s > 0 for s in (x_size, y_size, z_size)):
        raise ValueError("x_size, y_size, and z_size must be > 0")
    for k in ("XC", "YC", "ZC"):
        if k not in df.columns:
            raise KeyError(f'missing required column "{k}"')

    # Fast views
    x = df["XC"].to_numpy(dtype=np.float32, copy=False)
    y = df["YC"].to_numpy(dtype=np.float32, copy=False)
    z = df["ZC"].to_numpy(dtype=np.float32, copy=False)

    xmin, xmax = float(np.nanmin(x)), float(np.nanmax(x))
    ymin, ymax = float(np.nanmin(y)), float(np.nanmax(y))
    zmin, zmax = float(np.nanmin(z)), float(np.nanmax(z))
    ov = float(dist)

    # Build non-overlapping core edges in each axis: [lo, hi)
    def edges(lo, hi, step):
        e = []
        cur = lo
        while cur <= hi:
            nxt = cur + step
            e.append((cur, nxt))
            cur = nxt
        return e

    x_edges = edges(xmin, xmax, x_size)
    y_edges = edges(ymin, ymax, y_size)
    z_edges = edges(zmin, zmax, z_size)

    rows = []
    tile_id = 0
    for ix, (x_lo, x_hi) in enumerate(x_edges):
        # Precompute masks by axis for speed
        cxm_x = (x >= x_lo)   & (x < x_hi)
        exm_x = (x >= x_lo - ov) & (x < x_hi + ov)
        for iy, (y_lo, y_hi) in enumerate(y_edges):
            cxm_xy = cxm_x & ((y >= y_lo) & (y < y_hi))
            exm_xy = exm_x & ((y >= y_lo - ov) & (y < y_hi + ov))
            for iz, (z_lo, z_hi) in enumerate(z_edges):
                tile_id += 1
                core_mask = cxm_xy & ((z >= z_lo) & (z < z_hi))
                exp_mask  = exm_xy & ((z >= z_lo - ov) & (z < z_hi + ov))

                n_smoothed = int(np.count_nonzero(core_mask))
                if n_smoothed == 0:
                    # Still record the tile (keeps grid completeness)
                    n_indexed = int(np.count_nonzero(exp_mask))
                    rows.append({
                        "tile_id": tile_id, "ix": ix, "iy": iy, "iz": iz,
                        "x_lo": float(x_lo), "x_hi": float(x_hi),
                        "y_lo": float(y_lo), "y_hi": float(y_hi),
                        "z_lo": float(z_lo), "z_hi": float(z_hi),
                        "x_center": float((x_lo + x_hi) * 0.5),
                        "y_center": float((y_lo + y_hi) * 0.5),
                        "z_center": float((z_lo + z_hi) * 0.5),
                        "cx": np.nan, "cy": np.nan, "cz": np.nan,
                        "n_indexed": n_indexed,
                        "n_smoothed": 0,
                    })
                    continue

                n_indexed = int(np.count_nonzero(exp_mask))

                # Core centroid (mean of points within the core)
                cx_mean = float(np.nanmean(x[core_mask]))
                cy_mean = float(np.nanmean(y[core_mask]))
                cz_mean = float(np.nanmean(z[core_mask]))

                rows.append({
                    "tile_id": tile_id, "ix": ix, "iy": iy, "iz": iz,
                    "x_lo": float(x_lo), "x_hi": float(x_hi),
                    "y_lo": float(y_lo), "y_hi": float(y_hi),
                    "z_lo": float(z_lo), "z_hi": float(z_hi),
                    "x_center": float((x_lo + x_hi) * 0.5),
                    "y_center": float((y_lo + y_hi) * 0.5),
                    "z_center": float((z_lo + z_hi) * 0.5),
                    "cx": cx_mean, "cy": cy_mean, "cz": cz_mean,
                    "n_indexed": n_indexed,
                    "n_smoothed": n_smoothed,
                })

    return pd.DataFrame(rows)

In [ ]:
res = summarize_bands_xyz(
    bm, dist=50, x_size=50, y_size=50, z_size=50
)

print()
sns.histplot(res, x="n_indexed", bins=20)

In [ ]:
# col = "CATE"
# dist = 20
# out_col = f"{col}_SUAV"

# suavizar_col(bm, col, dist, out_col)

In [ ]:
col = "CATE"
dists = [10, 20]

out_col = f"{col}_SUAV"
out_cols = [f"{out_col}_{d}" for d in dists]

suavizar_batched_xyz_multi_stable(
    bm, col, dists, out_cols, 
    x_size=50, y_size=50, z_size=50, 
)

In [ ]:
report = report_volume_variation(bm, "VOL", col, "S10")
report

In [ ]:
c = 612.0
lt = 10
d = 20

bm[(c-lt/2-d <= bm["ZC"]) & (bm["ZC"]<c+lt/2+d)].shape

In [ ]:
bm[(c-lt/2 <= bm["ZC"]) & (bm["ZC"]<c+lt/2)].shape